
# Calculate Detector Counts: NEI
Compute the AIA response use our full emission model, including non-equilibrium ionization.

In [ ]:
import os
import time
import h5py

import numpy as np
import astropy.units as u
import sunpy.sun.constants
import matplotlib.pyplot as plt
import dask
import distributed

import synthesizAR
from synthesizAR.instruments import InstrumentSDOAIA
from synthesizAR.atomic import EmissionModel, Element
from synthesizAR.interfaces import EbtelInterface

%matplotlib inline

In [ ]:
client = distributed.Client()
client

Load in the desired field and emission model

In [ ]:
field = synthesizAR.Field.restore('/storage-home/w/wtb2/data/timelag_synthesis_v2/cooling/field_checkpoint/')

We are using an emission model which includes only the most dominant ions. Comparisons to the temperature response functions show these provide accurate coverage.

In [ ]:
em_model = EmissionModel.restore('/storage-home/w/wtb2/data/timelag_synthesis_v2/base_emission_model.json')

Compute and store the non-equilibrium ionization populations for each loop

In [ ]:
futures = em_model.calculate_ionization_fraction(field,
                '/storage-home/w/wtb2/data/timelag_synthesis_v2/cooling/nei/ionization_fractions.h5',
                interface=EbtelInterface, client=client)

In [ ]:
em_model.save('/storage-home/w/wtb2/data/timelag_synthesis_v2/cooling/nei/emission_model.json')

In [ ]:
futures = None

Or just reload the emission model

In [ ]:
em_model = EmissionModel.restore('/storage-home/w/wtb2/data/timelag_synthesis_v2/cooling/nei/emission_model.json')

Compute the detector counts

In [ ]:
aia = InstrumentSDOAIA([0,10000]*u.s, use_temperature_response_functions=False)

In [ ]:
observer = synthesizAR.Observer(field,[aia],parallel=True)

In [ ]:
observer.build_detector_files('/storage-home/w/wtb2/data/timelag_synthesis_v2/cooling/nei/',
                              ds=0.5*u.Mm)

In [ ]:
tasks = observer.flatten_detector_counts(emission_model=em_model)

In [ ]:
tasks['SDO_AIA_parameters'].compute()

In [ ]:
tasks['SDO_AIA_counts'].compute(resources={'MEMORY':'100GB'})

And finally build the maps

In [ ]:
tasks = observer.bin_detector_counts('/storage-home/w/wtb2/data/timelag_synthesis_v2/cooling/nei/')

In [ ]:
jobs = client.compute(tasks['SDO_AIA'])

In [ ]:
jobs = None